# Obtain the albedo at SIGMA-A and SIGMA-B

Shunan Feng

In [ ]:
import geemap
import ee
import pandas as pd
import altair as alt
import geemap.colormaps as cm
# import numpy as np
# import os

# SIGMA-A
<img src="http://globalcryospherewatch.org/cryonet/questionnaire/files/site_map.62.png" alt="Matlab" width="300" height="300"/>

In [ ]:
# 
Map = geemap.Map(basemap='HYBRID')
Map

In [ ]:
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard by https://github.com/guiattard
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

## MODIS

In [ ]:

# aoi = ee.Geometry.Point([dfsYear.lon[i], dfsYear.lat[i]]).buffer(300)
aoi = ee.Geometry.Point( -67.6283,  78.0517)
Map.addLayer(aoi, {}, 'SIGMA_A')
Map.centerObject(aoi, 8)
# aoi = ee.FeatureCollection([
#     ee.Feature(ee.Geometry.Point(
#         [
#             dfsYear.lon[i], dfsYear.lat[i]
#         ]
#     ), {'label': dfsYear.Year[i]})
# ])
# date_start = str(dfsYear.Year[i]) + '-' + str(1) + '-' + str(1) 
# date_end = str(dfsYear.Year[i]) + '-' + str(12) + '-' + str(31) 
# print(date_start)
# create filter for image collection

colFilter = ee.Filter.And(
    # ee.Filter.bounds(aoi),
    # ee.Filter.intersects('.geo', aoi),
    ee.Filter.geometry(aoi),
    # ee.Filter.date(date_start, date_end)
)

# MOD10A1.006 Terra Snow Cover Daily Global 500m
modisCol = ee.ImageCollection('MODIS/006/MOD10A1').filter(colFilter)

# if multiSat.size().getInfo()==0:
#     continue

pointValue = modisCol.getRegion(aoi, 500).getInfo() # 300 is the buffer radius
df = ee_array_to_df(pointValue, ['Snow_Albedo_Daily_Tile', 'NDSI_Snow_Cover'])
df['Snow_Albedo_Daily_Tile'] = df['Snow_Albedo_Daily_Tile'] / 100
df['NDSI_Snow_Cover'] = df['NDSI_Snow_Cover'] / 100

# df.to_csv('sigmaAmodis.csv', index=False)


# pointValueFile = 'promice/' + stationName + '_' + str(dfsYear.Year[i]) + '.csv'

# all_list = all_list.map(func_bbz)
# work_dir = os.path.expanduser('~/Downloads')
# out_csv = os.path.join(work_dir, 'landsat.csv')
# out_csv = os.path.join('landsat.csv')
# geemap.extract_values_to_points(aoi, multiSat, out_csv)


In [ ]:
df['year'] = df['datetime'].dt.year
df['doy'] = df['datetime'].dt.dayofyear

selection = alt.selection_multi(fields=['year'], bind='legend')

albedoChart1 = alt.Chart(
    df,
    title="MODIS Albedo"
    ).mark_line(
    color='red',
    size=3
).transform_window(
    rolling_mean='mean(Snow_Albedo_Daily_Tile)',
    frame=[-15, 15]
).encode(
    x='doy:O',
    y='rolling_mean:Q',
    color='year:O',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    width=600, height=300
).interactive()

In [ ]:
albedoChart1

In [ ]:
# df['albedo_moveave'] = df['Snow_Albedo_Daily_Tile'].rolling(window=15).mean()

# index = df['NDSI_Snow_Cover'] < 80
df['snowcover'] = 0
df['snowcover'] = df['snowcover'].mask(df['Snow_Albedo_Daily_Tile'] < 80, 1)

In [ ]:
alt.Chart(
    df,
    title="Snow free days of month (albedo < 0.8)"
).mark_rect().encode(
    x='year(datetime):O',
    y='month(datetime):O',
    color=alt.Color('sum(snowcover):Q', scale=alt.Scale(scheme="inferno")),
    tooltip=[
        # alt.Tooltip('monthdate(datetime):T', title='Date'),
        alt.Tooltip('sum(snowcover):Q', title='days without snow')
    ]
).properties(width=550)

## Multisat (Landsat and Sentinel-2)

In [ ]:
rmaCoefficients = {
  'itcpsL7': ee.Image.constant([0.0156, 0.0013, 0.0081, 0.0034, -0.0021, 0.0011]) \
          .multiply(10000),
  'slopesL7': ee.Image.constant([0.9823, 1.0096, 0.9918, 0.9979, 0.8944, 1.1510]),
  'itcpsS2': ee.Image.constant([-0.0039, -0.0082, -0.0073, -0.0790, -0.0038, 0.0020]) \
          .multiply(10000),
  'slopesS2': ee.Image.constant([1.0246, 1.0204, 1.0328, 1.1107, 1.0338, 1.0012])
}; #rma

In [ ]:
# Function to get and rename bands of interest from OLI.
def renameOli(img):
  return img.select(
    ['B2',   'B3',    'B4',  'B5',  'B6',    'B7',    'pixel_qa', 'radsat_qa'],
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa', 'radsat_qa'])

# Function to get and rename bands of interest from ETM+, TM.
def renameEtm(img):
  return img.select(
    ['B1',   'B2',    'B3',  'B4',  'B5',    'B7',    'pixel_qa', 'radsat_qa'],
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa', 'radsat_qa'])

# Function to get and rename bands of interest from Sentinel 2.
def renameS2(img):
  return img.select(
    ['B2',   'B3',    'B4',  'B8',  'B11',   'B12',   'QA60'],
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'QA60']
  )

def etm2oli(img):
  return img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) \
    .multiply(rmaCoefficients["slopesL7"]) \
    .add(rmaCoefficients["itcpsL7"]) \
    .round() \
    .toShort() 
    # .addBands(img.select('pixel_qa'))

def s22oli(img):
  return img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) \
    .multiply(rmaCoefficients["slopesS2"]) \
    .add(rmaCoefficients["itcpsS2"]) \
    .round() \
    .toShort() # convert to Int16
    # .addBands(img.select('pixel_qa'))
# def etm2oli(img):
#   return img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']).toShort() 
#     # .addBands(img.select('pixel_qa'))

# def s22oli(img):
#   return img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']).toShort() # convert to Int16
#     # .addBands(img.select('pixel_qa'))

def imRangeFilter(image):
  mask = image.lt(10000)
  return image.updateMask(mask)

# cloud mask for Landsat data based on fmask (pixel_qa)
def bitwiseExtract(value, fromBit, toBit):
  if (toBit == "undefined"):
      toBit = fromBit
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return value.rightShift(fromBit).bitwiseAnd(mask)
 #Daniel Wiell https:#gis.stackexchange.com/questions/363929/how-to-apply-a-bitmask-for-radiometric-saturation-qa-in-a-image-collection-eart
#*
# Function to mask clouds based on the pixel_qa band of Landsat 8 SR data.
# @param {ee.Image} image input Landsat 8 SR image
# @return {ee.Image} cloudmasked Landsat 8 image
#
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively. Bits 2 are water.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # waterBitMask = (1 << 2)
  # Get the pixel QA band.
  qa = image.select('pixel_qa')
  radsatQA = image.select('radsat_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  anySaturated = bitwiseExtract(radsatQA, 1, 11)
  saturateMask = anySaturated.Not()
  return image.updateMask(mask).updateMask(saturateMask)

#*
# Function to mask clouds based on the pixel_qa band of Landsat SR data.
# @param {ee.Image} image Input Landsat SR image
# @return {ee.Image} Cloudmasked Landsat image
#
def cloudMaskL457(image):
  qa = image.select('pixel_qa')
  radsatQA = image.select('radsat_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
  # water = qa.bitwiseAnd(1 << 2)
  # Remove edge pixels that don't occur in all bands
  mask2 = image.mask().reduce(ee.Reducer.min())
  anySaturated = bitwiseExtract(radsatQA, 1, 7)
  saturateMask = anySaturated.Not()
  return image.updateMask(cloud.Not()).updateMask(mask2).updateMask(saturateMask)

#*
 # Function to mask clouds using the Sentinel-2 QA band
 # @param {ee.Image} image Sentinel-2 image
 # @return {ee.Image} cloud masked Sentinel-2 image
 #
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  # Bits 1 is saturated or defective pixel
  saturateBitMask = 1 << 1
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0)) \
      .And(qa.bitwiseAnd(saturateBitMask).eq(0))

  return image.updateMask(mask)


In [ ]:
# Define function to prepare OLI images.
def prepOli(img):
  orig = img
  img = renameOli(img)
  img = maskL8sr(img)
  img = imRangeFilter(img)
#   img = addAlbedo(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()))

# Define function to prepare ETM+/TM images.
def prepEtm(img):
  orig = img
  img = renameEtm(img)
  img = cloudMaskL457(img)
  img = etm2oli(img)
  img = imRangeFilter(img)
#   img = addAlbedo(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()))

# Define function to prepare S2 images.
def prepS2(img):
  orig = img
  img = renameS2(img)
  img = maskS2clouds(img)
  img = s22oli(img)
  img = imRangeFilter(img)
#   img = addAlbedo(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()).set('SATELLITE', 'SENTINEL_2'))


In [ ]:


# aoi = ee.Geometry.Point([dfsYear.lon[i], dfsYear.lat[i]]).buffer(300)
aoi = ee.Geometry.Point( -67.6283,  78.0517)
Map.addLayer(aoi, {}, 'SIGMA_A')
# aoi = ee.FeatureCollection([
#     ee.Feature(ee.Geometry.Point(
#         [
#             dfsYear.lon[i], dfsYear.lat[i]
#         ]
#     ), {'label': dfsYear.Year[i]})
# ])
# date_start = str(dfsYear.Year[i]) + '-' + str(1) + '-' + str(1) 
# date_end = str(dfsYear.Year[i]) + '-' + str(12) + '-' + str(31) 
# print(date_start)
# create filter for image collection

colFilter = ee.Filter.And(
    # ee.Filter.bounds(aoi),
    # ee.Filter.intersects('.geo', aoi),
    ee.Filter.geometry(aoi),
    # ee.Filter.date(date_start, date_end),
    # ee.Filter.calendarRange(5, 9, 'month'),
    ee.Filter.lt('CLOUD_COVER', 50),
    ee.Filter.lte('GEOMETRIC_RMSE_MODEL', 30),
    # ee.Filter.gt('SUN_ELEVATION', 5),
    ee.Filter.Or(
        ee.Filter.eq('IMAGE_QUALITY', 9),
        ee.Filter.eq('IMAGE_QUALITY_OLI', 9)
    )
)

s2colFilter =  ee.Filter.And(
    # ee.Filter.bounds(aoi),
    # ee.Filter.intersects('.geo', aoi),
    ee.Filter.geometry(aoi),
    # ee.Filter.date(date_start, date_end),
    # ee.Filter.calendarRange(5, 9, 'month'),
    ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)
)

# oliCol = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
oliCol = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filter(colFilter) \
            .map(prepOli) \
            .select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
# etmCol = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA")
etmCol = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
            .filter(colFilter) \
            .map(prepEtm) \
            .select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
tmCol = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
            .filter(colFilter) \
            .map(prepEtm) \
            .select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
s2Col = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filter(s2colFilter) \
            .map(prepS2) \
            .select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
# landsatCol = etmCol.merge(tmCol)
landsatCol = oliCol.merge(etmCol).merge(tmCol)
multiSat = landsatCol.merge(s2Col).sort('system:time_start')

# if multiSat.size().getInfo()==0:
#     continue



### 90 m

In [ ]:
pointValue = multiSat.getRegion(aoi, 90).getInfo() # 300 is the buffer radius
df = ee_array_to_df(pointValue, ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) 

df['albedo'] = 0.356 * df.Blue/10000 + 0.13 * df.Red/10000 + 0.373 * df.NIR/10000 + 0.085 * df.SWIR1/10000 + 0.072 * df.SWIR2/10000 - 0.018
# df['albedo'] = 0.6002 * df.Blue/10000  + 1.7529 * df.Green/10000 - 2.7452 * df.Red/10000 + 1.1771 * df.NIR/10000 - 0.0376 * df.SWIR1/10000 + 0.0387 * df.SWIR2/10000 + 0.1231
df['ndsi'] = (df.Green - df.SWIR1) / (df.Green + df.SWIR1)
# df.to_csv('sigmaAmult90.csv', index=False)

alt.Chart(
    df,
    title = 'MultiSat Albedo 90m'
    ).mark_point().encode(
    x='datetime:T',
    y='albedo:Q',
    # color='Origin:N',
    # href='url:N',
    tooltip=['datetime', 'albedo', 'ndsi']
).properties(
    width=600, height=200
).interactive()

In [ ]:
df['year'] = df['datetime'].dt.year
df['doy'] = df['datetime'].dt.dayofyear

df['snowcover'] = 0
df['snowcover'] = df['snowcover'].mask(df['albedo'] < 0.8, 1)


alt.Chart(
    df,
    title="Percentage of snow free days (albedo < 0.8)"
).mark_rect().encode(
    x='year(datetime):O',
    y='month(datetime):O',
    color=alt.Color('mean(snowcover):Q', scale=alt.Scale(scheme="inferno")),
    tooltip=[
        # alt.Tooltip('monthdate(datetime):T', title='Date'),
        alt.Tooltip('mean(snowcover):Q', title='days without snow')
    ]
).properties(width=550)

# SIGMA-B
<img src="http://globalcryospherewatch.org/cryonet/questionnaire/files/site_map.61.png" alt="Matlab" width="300" height="300"/>


## MODIS

In [ ]:
# 
# aoi = ee.Geometry.Point([dfsYear.lon[i], dfsYear.lat[i]]).buffer(300)
aoi = ee.Geometry.Point( -69.0619,  77.5183)
Map.addLayer(aoi, {}, 'SIGMA_B')
# Map.centerObject(aoi, 10)
# aoi = ee.FeatureCollection([
#     ee.Feature(ee.Geometry.Point(
#         [
#             dfsYear.lon[i], dfsYear.lat[i]
#         ]
#     ), {'label': dfsYear.Year[i]})
# ])
# date_start = str(dfsYear.Year[i]) + '-' + str(1) + '-' + str(1) 
# date_end = str(dfsYear.Year[i]) + '-' + str(12) + '-' + str(31) 
# print(date_start)
# create filter for image collection

colFilter = ee.Filter.And(
    # ee.Filter.bounds(aoi),
    # ee.Filter.intersects('.geo', aoi),
    ee.Filter.geometry(aoi),
    # ee.Filter.date(date_start, date_end)
)

# MOD10A1.006 Terra Snow Cover Daily Global 500m
modisCol = ee.ImageCollection('MODIS/006/MOD10A1').filter(colFilter)

# if multiSat.size().getInfo()==0:
#     continue

pointValue = modisCol.getRegion(aoi, 500).getInfo() # 300 is the buffer radius
df = ee_array_to_df(pointValue, ['Snow_Albedo_Daily_Tile', 'NDSI_Snow_Cover'])
df['Snow_Albedo_Daily_Tile'] = df['Snow_Albedo_Daily_Tile'] / 100
df['NDSI_Snow_Cover'] = df['NDSI_Snow_Cover'] / 100

# df.to_csv('sigmaBmodis.csv', index=False)


# pointValueFile = 'promice/' + stationName + '_' + str(dfsYear.Year[i]) + '.csv'

# all_list = all_list.map(func_bbz)
# work_dir = os.path.expanduser('~/Downloads')
# out_csv = os.path.join(work_dir, 'landsat.csv')
# out_csv = os.path.join('landsat.csv')
# geemap.extract_values_to_points(aoi, multiSat, out_csv)


In [ ]:
df['year'] = df['datetime'].dt.year
df['doy'] = df['datetime'].dt.dayofyear

selection = alt.selection_multi(fields=['year'], bind='legend')

albedoChart1 = alt.Chart(
    df,
    title="MODIS Albedo"
    ).mark_line(
    color='red',
    size=3
).transform_window(
    rolling_mean='mean(Snow_Albedo_Daily_Tile)',
    frame=[-15, 15]
).encode(
    x='doy:O',
    y='rolling_mean:Q',
    color='year:O',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    width=600, height=300
).interactive()

In [ ]:
albedoChart1

In [ ]:
# df['albedo_moveave'] = df['Snow_Albedo_Daily_Tile'].rolling(window=15).mean()

# index = df['NDSI_Snow_Cover'] < 80
df['snowcover'] = 0
df['snowcover'] = df['snowcover'].mask(df['Snow_Albedo_Daily_Tile'] < 0.8, 1)

In [ ]:
alt.Chart(
    df,
    title="Snow free days of month (albedo < 0.8)"
).mark_rect().encode(
    x='year(datetime):O',
    y='month(datetime):O',
    color=alt.Color('sum(snowcover):Q', scale=alt.Scale(scheme="inferno")),
    tooltip=[
        # alt.Tooltip('monthdate(datetime):T', title='Date'),
        alt.Tooltip('sum(snowcover):Q', title='days without snow')
    ]
).properties(width=550)

## Multisat (Landsat and Sentinel-2)

### 90 m

In [ ]:
pointValue = multiSat.getRegion(aoi, 90).getInfo() # 300 is the buffer radius
df = ee_array_to_df(pointValue, ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) 

df['albedo'] = 0.356 * df.Blue/10000 + 0.13 * df.Red/10000 + 0.373 * df.NIR/10000 + 0.085 * df.SWIR1/10000 + 0.072 * df.SWIR2/10000 - 0.018
# df['albedo'] = 0.6002 * df.Blue/10000  + 1.7529 * df.Green/10000 - 2.7452 * df.Red/10000 + 1.1771 * df.NIR/10000 - 0.0376 * df.SWIR1/10000 + 0.0387 * df.SWIR2/10000 + 0.1231
df['ndsi'] = (df.Green - df.SWIR1) / (df.Green + df.SWIR1)
# df.to_csv('sigmaBmult90.csv', index=False)

alt.Chart(
    df,
    title = 'MultiSat Albedo 90m'
    ).mark_point().encode(
    x='datetime:T',
    y='albedo:Q',
    # color='Origin:N',
    # href='url:N',
    tooltip=['datetime', 'albedo', 'ndsi']
).properties(
    width=600, height=200
).interactive()

In [ ]:
df['year'] = df['datetime'].dt.year
df['doy'] = df['datetime'].dt.dayofyear

df['snowcover'] = 0
df['snowcover'] = df['snowcover'].mask(df['albedo'] < 0.8, 1)


alt.Chart(
    df,
    title="Percentage of snow free days (albedo < 0.8)"
).mark_rect().encode(
    x='year(datetime):O',
    y='month(datetime):O',
    color=alt.Color('mean(snowcover):Q', scale=alt.Scale(scheme="inferno")),
    tooltip=[
        # alt.Tooltip('monthdate(datetime):T', title='Date'),
        alt.Tooltip('mean(snowcover):Q', title='days without snow')
    ]
).properties(width=550)


# Animation

In [ ]:
roi = ee.Geometry.Polygon(
        [[-70.91269278237388, 78.10503647733918],
          [-70.91269278237388, 77.41819685694047],
          [-65.99081778237388, 77.41819685694047],
          [-65.99081778237388, 78.10503647733918]])

In [ ]:
year_start = 2001 #  MODIS 2000-02-18T00:00:00 - Present
year_end = 2020
month_start = 1
month_end = 12

date_start = ee.Date.fromYMD(year_start, month_start, 1)
date_end = ee.Date.fromYMD(year_end, month_end, 31)
years = ee.List.sequence(year_start, year_end) # time range of years
months = ee.List.sequence(month_start, month_end)

In [ ]:
# /*
# Albedo_BSA_vis Black-sky albedo for visible brodband
# Albedo_BSA_nir Black-sky albedo for NIR broadband
# Albedo_BSA_shortwave Black-sky albedo for shortwave broadband
# replace BSA with WSA for white scky albedo
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD43A3
# */
colFilter = ee.Filter.And(
    # ee.Filter.bounds(aoi),
    # ee.Filter.intersects('.geo', aoi),
    ee.Filter.geometry(roi),
    ee.Filter.date(date_start, date_end)
)

# MOD10A1.006 Terra Snow Cover Daily Global 500m
dataset = ee.ImageCollection('MODIS/006/MOD10A1').filter(colFilter).select('Snow_Albedo_Daily_Tile')

# dataset = ee.ImageCollection('MODIS/006/MOD10A1') \
#             .select('Snow_Albedo_Daily_Tile') \
#             .filterDate(date_start, date_end)

def funcY(y):
    def funcM(m):
        vi = dataset.filter(ee.Filter.calendarRange(y, y, 'year')) \
                    .filter(ee.Filter.calendarRange(m, m, 'month')) \
                    .median().clip(roi).divide(100) 
        return vi.set('year', y) \
                 .set('month', m) \
                 .set('system:time_start', ee.Date.fromYMD(y, m, 1).millis())
    return months.map(funcM)

albedoMonthly = ee.ImageCollection.fromImages(years.map(funcY).flatten())

In [ ]:
dates = albedoMonthly.reduceColumns(ee.Reducer.toList(), ["system:time_start"]).get('list')
dates = pd.to_datetime(dates.getInfo(), unit='ms')
dates = pd.DataFrame(dates,columns =['time'])
dates = dates.time.dt.strftime('%Y-%m-%d')

In [ ]:
palette = cm.get_palette('viridis', n_class=10)
visParams = {'min': 0,  'max': 1, 'palette': palette}
Map.add_time_slider(albedoMonthly, visParams, labels=dates.to_list() , time_interval=1)

In [ ]:
Map.add_colorbar(visParams, label="albedo", orientation="horizontal", layer_name="albedo")